# **Notas Parcial 3**

### **Árboles de decisión**

Un árbol de decisión es un modelo que ayuda a realizar predicciones a través de la división de niveles. Es decir, cada nivel del árbol se enfoca en una característica diferente, y esto se clasifica mediante un umbral (dependiendo si es mayor o menor a este)

### Split para árboles de regresión

En el modelo de regresión con árboles de decisión, el algoritmo divide los datos (dependiendo del umbral) y busca minimiza la varianza en cada nodo. Esto se realiza a través de la siguiente ecuación: 

$$
\text{Reducción de varianza} = \text{Varianza total} - \left( \frac{N_1}{N} \times \text{Var}(G1) + \frac{N_2}{N} \times \text{Var}(G2) \right)
$$

Se calcula la reducción de la varianza para cada umbral, y elige el valor que maximice esta reducción. Se repite el proceso en cada nodo hasta cumplir con una condición de parada. 

### Árboles de clasificación

En el modelo de clasificación con árboles de decisión, el algoritmo divide los datos (dependiendo del umbral) y busca minimizar la impureza en cada nodo, esto ya sea con el índice Gini o Entropía. Esto se realiza a través de la siguiente ecuación:

$$
\text{Ganancia de Impureza} = \text{Impureza Nodo Padre} - \left( \frac{N_1}{N} \times \text{Impureza}(G1) + \frac{N_2}{N} \times \text{Impureza}(G2) \right)
$$

Se calcula la reducción de la impureza para cada umbral, y elige el valor que maximice esta reducción. Se repite el proceso en cada nodo hasta cumplir con una condición de parada (como max depth, mínimo de muestras, etc.)

### **Random Forest Regresión**

Random forest es un algoritmo donde se tiene un conjunto de árboles de decisión para buscar predecir algo, a través de la combinación de los resultados de todos los árboles. Se toman muestras aleatorias de los datos (bootstrap), y con cada muestra se entrenan los árboles de decisión. Cada uno de estos árboles realiza una predicción y se promedia esto y se obtiene el resultado.

**Esto sirve para obtener una predicción más precisa y evitar el overfitting.**

### **Gradient Boosting**

Es un modelo de predicción donde se netrena un conjunto de árboles de decisión. Se toma la predicción de un árbol de decisión inicial y se calculan los residuos (errores) de este, entre las predicciones y los valores reales. Luego, se entrena un nuevo árbol para predecir los residuos, se corrigen los errores y se actualiza la predicción anterior con la mejorada. Eso se realiza a través de un proceso iterativo, donde hay un factor de aprendizaje que ayuda a disminuir el error, y se obtiene el menor error posible, además, se suman las predicciones de los nuevos árboles. La predicción de cada árbol se calcula de la siguiente manera:

$$
\gamma_j = \frac{\sum \text{Residuals}}{\sum_{i \in R_j} \text{Previous probability} (1 - \text{Previous probability})}
$$

 En el caso de clasificación, se utilizan los log odds, que luego se convienten a probabilidades con la siguiente función sigmoide:

 $$p_i = \frac{1}{1 + e^{-F_{m}(x_i)}}$$


### **XGBoost**

XGBoost es un modelo donde se suman árboles de decisión para lograr mejorar las predicciones. 

$$
\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + f_t(x_i)
$$

Aquí, se tiene lo siguiente:
- $\hat{y}_i^{(t)}$ será la predicción
- $f_t(x_i)$ es el nuevo árbol que se entrena
- $\hat{y}_i^{(t-1)}$ es la predicción anterior

Esto se realiza de manera iterativa, es decir, se suma un árbol nuevo en cada paso. Esto se hace para minimizar la contribución marginal a la función de pérdida. Se busca reducir el error de los árboles anteriores. La función de pérdida se compone de dos partes (árboles anteriores y árbol nuevo), se debe de optimizar la parte del árbol nuevo, ya que la primera parte se utiliza como constante. Se llega a lo siguiente:

$$
\mathcal{L}^{(t)} =
\sum_{i=1}^n l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) + \sum_{k=1}^{t} \Omega(f_k)
$$

Aquí, se tiene que:

$$
\Omega(f) = \gamma T + \frac{1}{2} \lambda \sum_{j=1}^{T} w_j^2
$$

Es decir, $\Omega(f)$ es un factor de regularización que penaliza árboles más complejos y ayuda a evitar el overfitting. 

Se utiliza la expansión de Taylor de segundo orden para poder minimizar la pérdida de manera mucho más fácil y se manera cuadrática, porque es una ecuación compleja. La función de pérdida se queda de la siguiente manera (considerando Taylor, el gradiente y el hessiano): 

$$
\tilde{\mathcal{L}}^{(t)} = \sum_{i=1}^n \left[ g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2 \right] + \Omega(f_t)
$$

Con esto, a cada árbol  $f_t$ se le pone un valor constante de  $w_j$ a cada región $R_j$. Esto permite que la función de pérdida, ya simplificada, sea la siguiente: 

$$
\mathcal{L}^{(t)} \approx \sum_{j=1}^{T} \left[
G_j w_j + \frac{1}{2} (H_j + \lambda) w_j^2
\right] + \gamma T
$$

Para obtener el output value, se necesita minimizar por respecto a $w_j$, esto a través de una derivada e igualando a 0.

$$
\frac{d}{dw_j} \left( G_j w_j + \frac{1}{2} (H_j + \lambda) w_j^2 \right) = 0
$$

$$
\textbf{Output value} = w_j^* = -\frac{G_j}{H_j + \lambda}
$$

Con este valor de $w_j^*$, se sustituye para obtener el similarity score:

Sustituyendo:

$$
\mathcal{L}_j(w_j^*) =
G_j \left(-\frac{G_j}{H_j + \lambda} \right) +
\frac{1}{2}(H_j + \lambda) \left( \frac{G_j^2}{(H_j + \lambda)^2} \right)
$$

$$
= -\frac{G_j^2}{H_j + \lambda} + \frac{1}{2} \cdot \frac{G_j^2}{H_j + \lambda}
= -\frac{1}{2} \cdot \frac{G_j^2}{H_j + \lambda}
$$

$$
{
\text{Similarity Score} = \frac{G_j^2}{H_j + \lambda}
}
$$

Esto nos sirve para ver que tan buena es una hoja, y si es útil mantenerla o hacerle split.

Para saber si vale la pena dividir una hoja en dos, se calcula el gain del split

$$
\text{Gain} = \frac{1}{2} \left( \text{Similarity}_\text{izq} + \text{Similarity}_\text{der} - \text{Similarity}_\text{padre} \right) - \gamma
$$

Si este valor es positivo, se hace el split, y $\gamma$ será un hiperparámetro

La predicción final se obtiene sumando todos los árboles de la siguiente manera:

$$
\hat{y}_i = F_0(x_i) + \sum_{t=1}^{M} f_t(x_i)
$$

Cabe mencionar que si es clasificación, se aplica la función sigmoide para convertirlo a probabilidad:

$$
p_i = \frac{1}{1 + e^{-\hat{y}_i}}
$$

### **Feature Importance**

**¿Qué es feature importance?**

Feature importance es una técnica con la cual se evalua la importancia o que tan relevante es una variable para que el modelo haga sus predicciones


WEIGHT / SPLIT: cuantas veces se utiliza cada variable
GAIN: en promedio cuanto baja la función de pérdida cuando utilizas una variable (normalizado de 0 a 1)

LightGBM:

lgb.plot_importance(model, max_num_features = 10, importance_type='gain')
lgb.plot_importance(model, max_num_features = 10, importance_type='split')

XGBoost:

plot_importance(model, max_num_features=10, importance_type="weight")
plot_importance(model, max_num_features=10, importance_type="gain")

CatBoost:

df_importances = pd.DataFrame({
    'feature': model.feature_names_,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=False)

df_importances

FALTARÍ RANDOM FOREST


**Partial dependency plot**

El partial dependency plot muestra como cambia la predicción del modelo cuando una variable toma distintos valores y las otras variables se mantienen fijas


*EJEMPLO:*

imaginá que tenés un modelo de predicción de precio de casas, y hacés un PDP para la variable num_habitaciones. El gráfico te muestra:

"Si todas las casas tuvieran 2 habitaciones, ¿cuál sería el precio promedio según el modelo? ¿Y si todas tuvieran 3? ¿4?"

Esto te permite ver si, por ejemplo, el modelo aprende que más habitaciones = mayor precio, o si hay un punto donde el efecto se aplana o incluso baja.

### **Comparativa**

La estructura del árbol:

* XGBoost produce árboles más simétricos y balanceados.

* LightGBM produce árboles más profundos y desbalanceados si no se controla.

La precisión y riesgo de overfitting:

* Leaf-wise (LightGBM) puede encontrar mejores divisiones, pero se sobreajusta más fácil

* Level-wise (XGBoost) es más estable, pero a veces menos preciso.

Diferencias generales:
* light gbm no crece de manera ordenada, ya que hace un histograma de 250 bins y no uno por uno

* en catboost las ramas son iguales, lo cual hace que la prediccion sea rápida

* catboost es mejor cuando hay variables categóricas, ya que realiza categorical encoding 

* en catboost las ramas son iguales, hace que la prediccion sea rapida

* catboost: lo especial de esto es el encoding (categorical boosting) esta caracterizada en features categoricas

* catboost: promedio de manera secuencial

* lo que hace catboost es que el encoding de cada variable en cada punto tenga que ver con el promedio conocido hasta cierto momento




| Característica              | **XGBoost**                                                | **LightGBM**                                               | **CatBoost**                                                  |
|-----------------------------|------------------------------------------------------------|-------------------------------------------------------------|----------------------------------------------------------------|
| **Velocidad**               | Rápido, pero más lento que LightGBM y CatBoost             | 🔥 Muy rápido gracias a histogramas y leaf-wise growth      | Rápido, aunque un poco más lento que LightGBM                  |
| **Precisión**               | Alta                                                       | Alta, a veces mejor con buen tuning                         | Muy alta, especialmente con categóricas                        |
| **Variables categóricas**   | ❌ No las maneja (requiere encoding manual)                | ❌ No las maneja (requiere encoding manual)                 | ✅ Soporte nativo + regularización secuencial                  |
| **Uso de memoria**          | Moderado                                                   | ✅ Muy eficiente (binning)                                   | Similar a XGBoost                                              |
| **Manejo de missing values**| ✅ Automático                                               | ✅ Automático                                                | ✅ Automático                                                   |
| **Soporte GPU**             | ✅ Sí (bastante estable)                                   | ✅ Sí (muy rápido)                                           | ✅ Sí (algo más limitado)                                      |
| **Instalación**             | Fácil (`pip install xgboost`)                             | Fácil (`pip install lightgbm`)                              | Un poco más pesada (`pip install catboost`)                   |
| **Documentación**           | Excelente                                                  | Buena                                                       | Muy buena                                                     |
| **Interacción con sklearn** | Muy buena                                                  | Muy buena                                                   | Muy buena                                                     |
| **Tolerancia al orden**     | ✅ Neutral                                                  | ✅ Neutral                                                   | ⚠️ Sensible (por codificación secuencial)                      |


**XGBoost**:

Optimiza una función de pérdida regularizada (como logística + penalización).

Usa expansión de Taylor de segundo orden → usa derivadas de segundo orden (Hessiana).


**LightGBM**:

Histograma binning: agrupa valores continuos en bins → acelera entrenamiento.

Crece los árboles leaf-wise con "depth limit".


**CatBoost**:

Usa Ordered Boosting: al entrenar, simula que los datos vienen en orden temporal → evita "target leakage".

Maneja categóricas con combinaciones de variables y estadísticos del conjunto 


### **Prueba de poder**

Una prueba de poder nos permite saber cuantos datos se necesitan en un AB test para confiar en los resultados, es decir, para que realmente se vea un cambio o efecto en la muestra. 

La prueba Z nos ayuda saber si una media o proporción observada difere significativamente de un valor esperado

$H_0$: No hay diferencia significativa
$H_A$: Si hay diferencia significativa

Cuando p-value es menor a 0.05, se rechaza la hipótesis nula, es decir la media de la muestra es significativamente diferente a la media

Existen dos tipos de errores en la prueba de poder:
* Error tipo 1 $\alpha$: Falso positivo (rechazas $H_0$ cuando en realidad es verdadera)
* Error tipo 2 $\beta$: Falso negativo (no se rechaza $H_0$ cuando en realidad es $H_A$)

**Power Analysis (Análisis de Potencia)**
El power analysis es una técnica estadística utilizada para determinar el tamaño de muestra (N) necesario para detectar un efecto de interés con una probabilidad específica. También se puede usar para calcular la potencia de una prueba dada una muestra y un tamaño de efecto.

En términos simples, nos ayuda a responder preguntas como:

- ¿Cuántas observaciones necesito para detectar un efecto real con alta probabilidad?
- ¿Qué tan probable es que mi estudio detecte un efecto real si realmente existe?

### Elementos clave
- Potencia : (1 - $\beta$) Es la probabilidad de rechazar $H_0$ cuando $H_A$ es negativa (Error tipo II). Usualmente queremos tener $\beta$ en 80%
- Nivel de significancia: $\alpha$ Es la probabilidad de rechazar $H_0$ cuando $H_0$ es verdadera (error tipo I)

**LO MATEMÁTICO Y EL CÓDIGO ESTÁ EN EL ARCHIVO PRUEBA_DE_PODER**
